# Create My Own Sound Collection

In this notebook, we will build our own collection of sound fonts by querying the Freesound API. We will use modular code stored in the **src** folder to:
- Query Freesound for sounds matching our desired keywords.
- Download high-quality previews.
- Store relevant metadata in a CSV file.

This approach follows the structure of the original Notebook 1 from the AMPLAB module, our implementation is modular.



## Environment Setup and Imports

This cell confirms that our configuration (API key, file paths, etc.) is loaded correctly from src/config.py and that our functions for querying Freesound are accessible.

In [1]:
# Ensure that your virtual environment is activated and dependencies are installed (see requirements.txt).

# Import configuration settings and utility modules from src
import os
import pandas as pd
import sys

# Añadir la ruta absoluta del directorio raíz del proyecto
sys.path.append(os.path.abspath(".."))

# Our configuration file contains API keys, file paths, etc.
from src import config  
# Our sound_collection module includes functions to query Freesound and process sound records.
from src.sound_collection import query_freesound, download_sound_preview, make_metadata_record

# Display configuration to confirm settings
print("Freesound API Key:", config.FREESOUND_API_KEY)
print("Files Directory:", config.RAW_DIR)
print("Metadata CSV File:", config.DATAFRAME_FILENAME)


Freesound API Key: zr5kUObkkKtoSIiWGRPG6DPNUMOxdU1ercdOGcaJ
Files Directory: ../data/raw
Metadata CSV File: ../data/metadata/fonts_collection.csv


## Prepare the Data Directory (Code)

In [2]:
# Check if the directory to store downloaded sound files exists; if not, create it.
for folder in [config.RAW_DIR, config.PROCESSED_DIR, config.METADATA_DIR]:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created directory: {folder}")
    else:
        print(f"Directory {folder} already exists.")


Directory ../data/raw already exists.
Directory ../data/processed already exists.
Directory ../data/metadata already exists.


## Define Freesound Query Parameters

Adjust the queries to target the type of sound “fonts” you want. For example, “dog bark” might be used as a percussive element, while “vowels” can add a human quality. The filters ensure that only sounds with appropriate durations are retrieved.

In [3]:
# Define a list of queries to build our sound collection.
# Here we customize our search terms and filters based on our creative vision.
# Define a new set of queries to capture electronic sounds, percussions, violins, basses, and synthesizers.
freesound_queries = [
    {
        'query': 'electronic soundscapes',
        'filter': 'duration:[5 TO 30]',  # adjust duration as needed
        'num_results': 20,
    },
    {
        'query': 'percussion loops',
        'filter': 'duration:[1 TO 5]',
        'num_results': 20,
    },
    {
        'query': 'violin solo',
        'filter': 'duration:[3 TO 10]',
        'num_results': 20,
    },
    {
        'query': 'bass groove',
        'filter': 'duration:[3 TO 10]',
        'num_results': 20,
    },
    {
        'query': 'synthesizer pad',
        'filter': 'duration:[5 TO 20]',
        'num_results': 20,
    },
]


# Display the queries for confirmation.
print("Freesound Queries:")
for q in freesound_queries:
    print(q)


Freesound Queries:
{'query': 'electronic soundscapes', 'filter': 'duration:[5 TO 30]', 'num_results': 20}
{'query': 'percussion loops', 'filter': 'duration:[1 TO 5]', 'num_results': 20}
{'query': 'violin solo', 'filter': 'duration:[3 TO 10]', 'num_results': 20}
{'query': 'bass groove', 'filter': 'duration:[3 TO 10]', 'num_results': 20}
{'query': 'synthesizer pad', 'filter': 'duration:[5 TO 20]', 'num_results': 20}


## Query Freesound and Collect Sound Objects 

This cell uses our query_freesound function to perform the API searches. The results from each query are concatenated into one list representing our overall collection.

In [4]:
# Initialize an empty list to store all retrieved sound objects.
all_sounds = []

# Loop through each query configuration and perform the Freesound search.
for query_info in freesound_queries:
    results = query_freesound(query_info['query'], query_info['filter'], query_info['num_results'])
    print(f"Retrieved {len(results)} sounds for query '{query_info['query']}'.")
    all_sounds.extend(results)

print(f"Total sounds retrieved: {len(all_sounds)}")


Retrieved 20 sounds for query 'electronic soundscapes'.
Retrieved 20 sounds for query 'percussion loops'.
Retrieved 8 sounds for query 'violin solo'.
Retrieved 20 sounds for query 'bass groove'.
Retrieved 20 sounds for query 'synthesizer pad'.
Total sounds retrieved: 88


## Download Sound Previews

In [5]:
# Loop through each sound and download its preview to our designated directory.
for idx, sound in enumerate(all_sounds):
    print(f"Downloading sound {idx+1}/{len(all_sounds)}: id {sound.id}")
    download_sound_preview(sound, config.RAW_DIR)


## Create Metadata Records and Save DataFrame

The make_metadata_record function extracts key details from each sound, and we then store the entire collection in a CSV file. This DataFrame will serve as the basis for further analysis and later stages of the project.

In [6]:
# Build a list of metadata records for each sound using our utility function.
metadata_records = [make_metadata_record(sound, config.RAW_DIR) for sound in all_sounds]

# Create a Pandas DataFrame from the metadata records.
df_metadata = pd.DataFrame(metadata_records)

# Save the DataFrame to CSV for later use.
df_metadata.to_csv(config.DATAFRAME_FILENAME, index=False)
print(f"Saved metadata DataFrame with {len(df_metadata)} entries to {config.DATAFRAME_FILENAME}.")


Saved metadata DataFrame with 88 entries to ../data/metadata/fonts_collection.csv.


## Display the Metadata DataFrame

Displaying the DataFrame allows you to visually inspect the metadata and confirm that all information (e.g., sound names, tags, file paths) has been captured correctly.

In [7]:
# Load the metadata DataFrame to verify its contents.
df_loaded = pd.read_csv(config.DATAFRAME_FILENAME)
display(df_loaded)


,name,username,license,tags,freesound_id,path
0,Data Corruption,staticpony1,http://creativecommons.org/publicdomain/zero/1.0/,"['Sounds', 'Electric', 'EDM', 'Soundscapes', '...",729455,../data/raw/729455_4508519-hq.ogg
1,20131203_Electronic Device Buzzing _ZoomH2nXY.wav,Soundscape_Leuphana,http://creativecommons.org/publicdomain/zero/1.0/,"['device', 'noise', 'electronic', 'electric', ...",209660,../data/raw/209660_3844637-hq.ogg
2,HUM 34,pabloperez,http://creativecommons.org/publicdomain/zero/1.0/,"['sound', 'oscillator', 'abstract', 'electroni...",403708,../data/raw/403708_55313-hq.ogg
3,HUM 33,pabloperez,http://creativecommons.org/publicdomain/zero/1.0/,"['sound', 'oscillator', 'abstract', 'electroni...",403709,../data/raw/403709_55313-hq.ogg
4,HUM 15,pabloperez,http://creativecommons.org/publicdomain/zero/1.0/,"['sound', 'oscillator', 'abstract', 'electroni...",403711,../data/raw/403711_55313-hq.ogg
...,...,...,...,...,...,...
83,scaryscape_scarydoublethrill.wav,suonho,https://creativecommons.org/licenses/by/4.0/,"['abstract', 'alien', 'ambience', 'ambient', '...",58958,../data/raw/58958_308-hq.ogg
84,Lounge Ambient Music Loop,orangefreesounds,http://creativecommons.org/licenses/by-nc/3.0/,"['music-loop', 'atmosphere', 'chil-out', 'ambi...",242080,../data/raw/242080_4388723-hq.ogg
85,130_C_pad_01,DigitalUnderglow,http://creativecommons.org/publicdomain/zero/1.0/,"['Retro', 'Synthwave', 'Pad', 'Keys', 'Synthes...",757714,../data/raw/757714_14904072-hq.ogg
86,"Bass, Pad & Piano.wav",D_A_G_A_Z,http://creativecommons.org/licenses/by/3.0/,"['Electric-Dance-Music', 'Bass', 'Synthesizer'...",369268,../data/raw/369268_6795322-hq.ogg
